<a href="https://colab.research.google.com/github/neohack22/IASD/blob/graphs/WS_22_01_DataLoadingAndExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

If you have not created your Neo4J Sandbox yet, please run the pre-setup notebook to do sp.

# Setup

Install the necessary library in your Colab notebook environment and connect to your hosted Neo4J Sandbox.

In [ ]:
!pip install neo4j

Copy the connection details that 

In [ ]:
ip = "54.174.38.179"
bolt_port = "7687"
username = "neo4j"
password = "spots-carrier-wires"

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://" + ip + ":" + bolt_port, auth=(username, password))

print(driver.address) # your-sandbox-ip:your-sandbox-bolt-port

54.174.38.179:7687


# Citation Dataset Loading

In this notebook we're going to load the citation dataset into Neo4j.

## Reset database
Ensure you start with a clean and empty database in your Sandbox (it may be pre-loaded with toy data). To do so, run the following cell to delete all nodes and relationships.



In [ ]:
query = """
  CALL apoc.periodic.iterate(
    "MATCH (n) RETURN n", 
    "DETACH DELETE n", 
    {batchSize:1000}
  )
  yield batches, total return batches, total
"""

with driver.session() as session:
  result = session.run(query)
  for row in result:
    print(row)

<Record batches=133 total=132259>


Let's show now how to load a JSON file, and display each record.

It is useful to expore records in order to understand theyr structure (name and type of attributes, etc. ). Of course you could use a schema inference tool, but this is beyond the scope of this lab. 

In [ ]:
query = """
CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/dblp-ref-0.json")
YIELD value WITH value
RETURN value
LIMIT 100;
"""



with driver.session() as session:
  result = session.run(query)
  for row in result:
    print(row)

Now, let's modify the above code in order to diplay records once some of the fields ("id", "authors", "references", "venue") have been discarded. 

In [ ]:
query = """
CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/dblp-ref-0.json")
YIELD value WITH  apoc.map.clean(value,["id","authors","references", "venue"],[]) AS filtered
RETURN filtered
LIMIT 100;
"""


with driver.session() as session:
  result = session.run(query)
  for row in result:
    print(row)

## Create Constraints

First let's create some constraints to make sure we won't import duplicate data. We basically use an attribute as an identifier for each type of node that we will create.

In [ ]:
with driver.session() as session:
    display(session.run("CREATE CONSTRAINT ON (a:Article) ASSERT a.index IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (v:Venue) ASSERT v.name IS UNIQUE").consume().counters)

## Load the data

Now let's load the data into the database. We will create nodes and relationships for *Articles*, *Venues*, and *Authors* from JSON files that represent an extract of the DBLP dataset. 

We will load/import 4 files so as to avoid overloading problems (othewise settings should be changed, typically the session timeout). 


In [ ]:
# first dataset

query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-0.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   RETURN value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=25 total=24666 timeTaken=34 committedOperations=24666 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 25, 'committed': 25, 'failed': 0, 'errors': {}} operations={'total': 24666, 'committed': 24666, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 158066, 'relationshipsCreated': 256228, 'nodesCreated': 158066, 'propertiesSet': 250075, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


In [ ]:
# second dataset

query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-1.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   RETURN value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=10 total=9506 timeTaken=12 committedOperations=9506 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 10, 'committed': 10, 'failed': 0, 'errors': {}} operations={'total': 9506, 'committed': 9506, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 42291, 'relationshipsCreated': 87333, 'nodesCreated': 42291, 'propertiesSet': 75929, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


In [ ]:
# third dataset


query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-2.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   RETURN value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=14 total=13641 timeTaken=16 committedOperations=13641 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 14, 'committed': 14, 'failed': 0, 'errors': {}} operations={'total': 13641, 'committed': 13641, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 53696, 'relationshipsCreated': 119954, 'nodesCreated': 53696, 'propertiesSet': 100724, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


In [ ]:
#fourth dataset


query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-3.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   RETURN value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=5 total=4143 timeTaken=3 committedOperations=4143 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 5, 'committed': 5, 'failed': 0, 'errors': {}} operations={'total': 4143, 'committed': 4143, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 10563, 'relationshipsCreated': 18924, 'nodesCreated': 10563, 'propertiesSet': 23610, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


Let's check/ensure that we did not create any *Article* without title.

In [ ]:
query = """
  call apoc.periodic.iterate(
    'MATCH (a:Article) WHERE not(exists(a.title)) RETURN a',
    'DETACH DELETE a',
    {batchSize:1000}
  )
  yield batches, total return batches, total
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=133 total=132357>


# Explore the data

Let's use the intuitive Neo4J Browser to explore the data that we have imported using Cypher queries. To access the Browser, go to your Sandbox control panel (at https://sandbox.neo4j.com/) and click the *Open* button.

Here are some queries left as exercise to explore your newly created DBLP database in the Neo4J Browser. Various solutions are valid for each question. Hint: use the "LIMIT" clause to avoid returning too many results and slowing down your queries.

- Show some *Articles* and their *Author*

- Show some articles that talk about "random forests". Hint: use their *title* and *abstract* properties.

- Get the total number of *Article*, *Author* and *Venue* in the database. Hint: use nodes' "LABEL".

- Get the number of relationships for each type (*VENUE*, *CITED*, ...). Hint: use relationships' "TYPE".

- Find the most cited *Article*.


- Find the Top 10 *Author*s with the most collaborations. We count one collaboration as one article written by several authors.

- Find the *Author* with whom "Salvatore Greco" has co-authored the most with.

How could we use this last query as a basis to recommend future collaborations to Salvatore Greco? To make this easier, we will need to infer a graph of co-authorship.